In [ ]:
   
        while self.state in accepted_states:
            msg = self.request_socket.recv_json()
            match msg['id']:
                case 'connect_request':
                    self.state = StateController.CONNECTED
                    _dict = {}
                    _dict['id']= 'connect_reply'
                    _dict['detectors'] = self.detector_options
                    _dict['magnifications'] = self.magnification_options
                    _dict['magnification'] = self.microscope.get_stem_magnification()
                    _dict['blanked'] = self.microscope.get_beam_blanked()
                    _dict['screen_current'] = self.microscope.get_screen_current()
                
                case 'scan_request':
                    msg.pop('id')
                    self.state = StateController.PROCESSING
                    while self.state != StateController.CONNECTED:
                        time.sleep(0.01)
                    self.set_scan_parameters(**msg)
                    _dict = {}
                    _dict['id'] = 'scan_reply'

                case 'move_request':
                    msg.pop('id')
                    self.move(**msg)
                    _dict = {}
                    _dict['id'] = 'move_reply'
                    _dict_pos = self.microscope.get_stage_position()
                    _dict['x'] = _dict_pos['x']
                    _dict['y'] = _dict_pos['y']
                    _dict['z'] = _dict_pos['z']
                    _dict['tilt'] = _dict_pos['a']

                case 'postion_request':
                    msg.pop('id')
                    _dict = {}
                    _dict['id'] = 'position_reply'
                    _dict_pos = self.microscope.get_stage_position()
                    _dict['x'] = _dict_pos['x']
                    _dict['y'] = _dict_pos['y']
                    _dict['z'] = _dict_pos['z']
                    _dict['tilt'] = _dict_pos['a']

                case 'blank_request':
                    msg.pop('id')
                    self.microscope.set_beam_blanked(msg['blanked'])
                    _dict = {}
                    _dict['id'] = 'blank_reply'
                    _dict['blanked'] = self.microscope.get_beam_blanked()

                case 'magnification_request':
                    msg.pop('id')
                    self.microscope.set_stem_magnification(msg['magnification'])
                    _dict = {}
                    _dict['id'] = 'magnification_reply'
                    _dict['magnification'] = self.microscope.get_stem_magnification()


                case 'defocus_request':
                    msg.pop('id')
                    if msg['defocus'] is not None:
                        self.microscope.set_defocus(msg['defocus'])
                    _dict = {}
                    _dict['id'] = 'defocus_reply'
                    _dict['defocus'] = self.microscope.get_defocus()
                case 'disconnect_request':
                    self.state = StateController.PROCESSING
                    while self.state != StateController.CONNECTED:
                        time.sleep(0.01)

                    self.state = StateController.IDLE
                    _dict = {}
                    _dict['id'] = 'disconnect_reply'

        self.request_socket.close()
        self.publisher_socket.close()
        self.context.term()       



In [ ]:
import zmq
import time

connection = "192.168.0.1"
request_port = 50000
publisher_port = 50001

context = zmq.Context()
request_socket = context.socket(zmq.REQ)
publisher_socket = context.socket(zmq.SUB)
request_socket.connect(f'tcp://{connection}:{request_port}')
publisher_socket.connect(f'tcp://{connection}:{publisher_port}')

msg = {
    'id': 'connect_request'
}
request_socket.send_json({'id': 'connect_request'})
reply = request_socket.recv_json()

print(reply)



In [ ]:

msg = {
    'id': 'scan_request',
    'isscan': False,
    'dwell_time': 1*10**-6,
    'frame_size': 1024,
    'scan_time': 0.63,
    'detectors': ['HAADF']
}
request_socket.send_json(msg)
reply = request_socket.recv_json()

print(reply)

In [3]:
from tomoacquire.registrations import TOMOACQUIRE_MICROSCOPES

TOMOACQUIRE_MICROSCOPES.help()

2025-04-08 18:17:35,537 - INFO - FEI: FEI, <class 'tomoacquire.microscopes.FEI.FEIMicroscope'>
2025-04-08 18:17:35,542 - INFO - TEMSCRIPT: TEMScript, <class 'tomoacquire.microscopes.temscript.TEMSCriptMicroscope'>


In [ ]:
import numpy as np
msg = {
    'id': 'move_request',
    'x': -6.34e-07,
    'y': None,
    'z': None,
    'tilt': None
}

request_socket.send_json(msg)
reply = request_socket.recv_json()
print(reply)

In [ ]:
msg = {
    'id': 'postion_request',
}
request_socket.send_json(msg)
reply = request_socket.recv_json()
print(reply)

In [ ]:
msg = {
    'id': 'defocus_request',
    'defocus': 2.3124891679928026e-05
}
request_socket.send_json(msg)
reply = request_socket.recv_json()
print(reply)